Adressa Dataset: Gulla, J. A., Zhang, L., Liu, P., Özgöbek, Ö., & Su, X. (2017, August). The Adressa dataset for news recommendation. In Proceedings of the International Conference on Web Intelligence (pp. 1042-1048). ACM.

In [1]:
import pandas as pd
from tqdm import tqdm
import os
import json
from utils.adressa_util import preprocessing
from pathlib import Path
#from utils.adressa_util.entity_linking import add_entities


In [2]:
one_week_path = './adressa/one_week'
content_news = './adressa/content_refine'
out_path = './adressa/mind_format'

In [3]:
hash_title, hash2id = preprocessing.news_title(one_week_path)

1513739it [00:23, 63522.66it/s]
1356987it [00:18, 72699.88it/s] 
1087223it [00:19, 56576.47it/s]
1327429it [00:24, 53741.60it/s]
1613128it [00:28, 57602.93it/s]
1648346it [00:29, 56668.17it/s]
1496417it [00:32, 46174.50it/s]


In [4]:
out_path = Path(out_path)
preprocessing.write_news_files(hash_title,hash2id,out_path)

100%|██████████| 20428/20428 [00:00<00:00, 742748.53it/s]


In [5]:
article_hash_content = {}
for filename in os.listdir(content_news):
    with open(os.path.join(content_news, filename), "r") as f:
        article = json.load(f)
        article_hash = article['id']
        article_dict = {d['field'] : d['value'] for d in article['fields']}
        article_hash_content[article_hash] = article_dict

In [6]:
from utils.adressa_util.preprocessing import write_news_files_full
write_news_files_full(hash_title,hash2id,out_path,article_hash_content)

100%|██████████| 20428/20428 [00:00<00:00, 355218.72it/s]


In [7]:
## DA eliminare

from utils.adressa_util.preprocessing import entities_news
out_path = './adressa/mind_format/train/news_new.tsv'
out_path = Path(out_path)
entities_list = entities_news(out_path)

In [8]:
# Aggiungere parte attributi

In [9]:
## FINE DA ELIMINARe

In [10]:
hash_title

{'9f3999bd1a1a8d67bcb073ad54840f15cb30f014': 'Se lesernes nyttårsbilder',
 'f21680b77830223807e4847b3307e6ffa1e175ed': 'Dette er noen av verdens kuleste passtempler',
 '2607fc7d7b4c0ede839a5ff6d499fa428237443e': '- Det blir fyrverkeri',
 '68d1503c73ad169dcfff48214fd0274c4d612e63': 'Her koker det over for Tønseth. Så stakk han fra stadion i sinne.',
 '29f080ad9ac28bacd74b67f912db8bfba1f60258': 'Trump ønsker sine fiender godt nytt år',
 'f2ce698b3daf00cfcac0d5279053c4da9de07a92': 'Slik blir ferieåret 2017',
 '045e279b5467a73a32dcf94e014fb750c229366d': 'Bussen står, men trikken går',
 '40d2ee3cfa2a7119b197e293353d0e2401782b1a': 'Måtte i oppvaskmøte etter å ha kritisert landslaget',
 '080b1bbe40a36dda8cf11482647b8b534fec2226': 'Én av tre nordmenn vil trene mer i 2017',
 'e58a1032bd59080a9a194e44caf6e33d97b17ca1': 'Rema reklamerer med bokstaven æ',
 '17f24670798d52486bfa50f017966c810eba66a3': 'Trondheim, fra havna mot Pirbadet',
 '6924146e66b3d614e5f88078e49aac9158d6ca44': 'Slik så det ut i

# Do not execute or it will download entire refined pretrained model 30+GB
out_path = './adressa/mind_format'
out_path = Path(out_path)
add_entities(out_path)

In [11]:
from utils.adressa_util.preprocessing import process_users
uid2index, user_info = process_users(one_week_path, hash2id)

1513739it [00:30, 50243.12it/s]
1356987it [00:20, 66252.52it/s]
1087223it [00:21, 51341.46it/s]
1327429it [00:27, 47526.67it/s]
1613128it [00:32, 50154.93it/s]
1648346it [00:33, 49787.35it/s]
1496417it [00:34, 42960.45it/s]


In [12]:
for uid in tqdm(user_info):
    user_info[uid].sort_click()

100%|██████████| 640503/640503 [00:11<00:00, 56726.84it/s]


In [16]:
from utils.adressa_util.preprocessing import construct_behaviors

train_lines = []
test_lines = []

# Randomly sample NEG_NUM negative impression for every positive behavior
NEG_NUM = 20
for uindex in tqdm(user_info):
    uinfo = user_info[uindex]
    click_news = uinfo.click_news
    train_news = uinfo.train_news
    test_news = uinfo.test_news
    train_user,test_user = construct_behaviors(hash_title, uindex, click_news, train_news, test_news, NEG_NUM)
    train_lines.extend(train_user)
    test_lines.extend(test_user)

100%|██████████| 640503/640503 [03:29<00:00, 3057.74it/s] 


In [14]:
train_lines

['null\t0\tnull\t1 1424 2062 2386 3564 3694 3987 4038 4209 4793 5138 4796 5472 5341 7202 5341 5006 6938 5611 5884 4780 4789 4738 4986 4944 5837 4682 4769 4724 4654\t6156-1 10934-0 18982-0 8202-0 16325-0 15654-0 6362-0 8395-0 13974-0 17781-0 9630-0 13932-0 18094-0 7444-0 4690-0 20228-0 12365-0 3779-0 17292-0 10600-0 11020-0\n',
 'null\t1\tnull\t2 2386 2248\t8452-1 10827-0 12370-0 8879-0 3586-0 19238-0 16217-0 15681-0 1320-0 12268-0 5590-0 3260-0 9827-0 3547-0 13313-0 1949-0 411-0 11638-0 13170-0 13124-0 13632-0\n',
 'null\t5\tnull\t1 3 407 1 2386 2293 18 2773 2773 5427 9479 4796 7516 4798 5310 6826 1123 5006 4705 4670 4682 7784 5634 7784 4661 2281 6576 7031\t5754-1 12099-0 6116-0 17319-0 15911-0 20039-0 18275-0 18146-0 17722-0 17579-0 12986-0 3626-0 14852-0 13279-0 6870-0 16103-0 20071-0 10938-0 6198-0 18136-0 15176-0\n',
 'null\t6\tnull\t1 464 2796 2773 2386 2248 3775 3564 3987 3694 4796 5006 5138 5006 6489 4796 4705 4774 4780 6170 5837 4944 4694 5634 4834 4725 4722\t4647-1 8242-0 1015

In [17]:
from sklearn.model_selection import train_test_split

test_split_lines, valid_split_lines = train_test_split(
    test_lines, test_size=0.2, random_state=2023
)
with open(out_path / "train" / "behaviors.tsv", "w", encoding="utf-8") as f:
    f.writelines(train_lines)

with open(out_path / "valid" / "behaviors.tsv", "w", encoding="utf-8") as f:
    f.writelines(valid_split_lines)

with open(out_path / "test" / "behaviors.tsv", "w", encoding="utf-8") as f:
    f.writelines(test_split_lines)

In [2]:
def entities_news(path_news):
    entities = set()
    # Read entities from train news
    with open(path_news) as fp:
        for line in fp:
            newsid, vert, subvert, title, abstract, url, entity_info_title, entity_info_abstract = line.strip().split(
                '\t')
            # Add entities in the title
            for entity in eval(entity_info_title):
                entities.add(entity["WikidataId"])
            # Add entities in the abstract
            for entity in eval(entity_info_abstract):
                entities.add(entity["WikidataId"])
    return entities

def entities_existing(path_entity_emb):
    entities = set()
    with open(path_entity_emb) as fp:
        for line in fp:
            entity_id = line.strip().split('\t')[0]
            entities.add(entity_id)
    return entities
entities_new = entities_news('adressa/mind_format/train/news_new.tsv')
entities_old = entities_existing('data/train/entity_embedding.vec')
# Run a check to see that not all new entities are encoded

# Saving wiki id, encoded using same pipeline as jobs
pd.DataFrame(entities_new, columns=['entity']).to_csv('adressa/mind_format/train/old/wikidata_ids.txt', index=False, header=False)
# This part was implemented in notebook, TODO: Implement relation2vec
from utils.adressa_util.entity2vec import entities2vec
entities2vec(entities_new, out_path / "train" / "entity_embedding_.tsv")

In [29]:
#A Append entities to NO news
id_entities_ = {}
with open('adressa/mind_format/train/news_new.tsv', 'r') as eng_entities:
    for line in eng_entities:
        fields = line.split('\t')
        id_news = fields[0]
        entities_title = fields[-2]
        entities_abs = fields[-1]
        id_entities_[id_news] = [entities_title, entities_abs]
nor_news = []
with open('adressa/mind_format/train/news.tsv', 'r') as old_entities:
    for line in old_entities:
        fields = line.split('\t')
        id_news = fields[0]
        fields[-2] = id_entities_[id_news][0]
        fields[-1] = id_entities_[id_news][1]
        nor_news.append(fields)
with open('adressa/mind_format/train/news_no_eng_entities.tsv', "w") as new_file:
    for news in nor_news:
        new_file.write(("\t".join(news)))